In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pickle

import sys
sys.path.append('../../')
from Pulses import STA_pulses
from QEnvs.QEnvWave import QEnvWave

2023-06-16 19:10:36.060112: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 19:10:36.596297: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
episode_3_sta = np.load("Data/3/3_wpid/eval_best_episode.npy", allow_pickle=True)
episode_3_sta[2] = STA_pulses(episode_3_sta[3].shape[0], episode_3_sta[1][-1], episode_3_sta[1].shape[0] - 1, 1.).T

#populations_proportional_weights = [0., 0., 1.]
episode_3_wp = np.load("Data/3/3_wp/eval_best_episode.npy", allow_pickle=True)

#populations_proportional_weights = [-2., -1., 2.]
#populations_integral_weights = [0., -1., 0.]
#populations_derivative_weights = [0., 0., 30.]
episode_3_wpid = np.load("Data/3/3_wpid/eval_best_episode.npy", allow_pickle=True)

#decay_factors = [0., 1., 0.]
#populations_proportional_weights = [0., 0., 1.]
episode_3_me = np.load("Data/3/3_me/eval_best_episode.npy", allow_pickle=True)

episodes = [episode_3_sta, episode_3_wp, episode_3_wpid, episode_3_me]
episodes_me = [False, False, False, True]
episodes_names = ["Pure - STA", "Pure - P", "Pure - PID", "Mixed - P"]

In [3]:
def dummy_cost_function(env):
    return 0.0

def calculate_populations(env, pulse):
    times = env.times
    populations = []
    for state in env.run_qevolution(pulse):
        populations.append(np.abs(state) ** 2)
    return times, np.array(populations)

In [4]:
def simulate_episode(episode, me=False):
    num_steps = episode[1].shape[0] - 1
    time_max = episode[1][-1]
    num_qubits = episode[3].shape[0] 
    if me:
        num_qubits = num_qubits - 1
        pulses = episode[2][:-1].T
    else:
        pulses = episode[2].T

    env = QEnvWave(num_qubits=num_qubits,
                   time_max=time_max,
                   num_steps=num_steps - 1,
                   cost_function=dummy_cost_function)
    times, populations = calculate_populations(env, pulses)
    return times, pulses, populations.reshape((num_steps, num_qubits))

In [5]:
times = []
pulses = []
populations = []
for episode, me in zip(episodes, episodes_me):
    t, p, pop = simulate_episode(episode, me)
    times.append(t)
    pulses.append(p)
    populations.append(pop)

In [6]:
# Save
results = [episodes_names, times, pulses, populations]
with open("Data/3_states_comparison.pkl", "wb") as f:
    pickle.dump(results, f)